# Inference example with a Cellpose model: www.cellpose.org
The model is based on U-Net, however rather than training it directly on bitmask targets they first convert them to "spatial flows" representations and train on that. This makes segmentation of dense and touching cells more reliable. For details and additional tricks they use see the paper "Cellpose: a generalist algorithm for cellular segmentation".

To train it I used the script provided in the cellpose repo ie: `python -m cellpose --train ...` after I converted the dataset to the input format it expects.

In inference I just submit the masks as they were returned from the model - no postprocessing

In [ ]:
!pip install --no-index ../input/cellposewheels/cellpose-0.7.2-py3-none-any.whl --find-links=../input/cellposewheels

#### I'm running it in a separate process rather than as a regular notebook because I've faced issues with numpy version not updating after the `pip install` above. If you know how to fix this please let me know.

In [ ]:
%%writefile run.py
import numpy as np
from cellpose import models, io, plot
from pathlib import Path
import pandas as pd

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

test_dir = Path('../input/sartorius-cell-instance-segmentation/test')
test_files = [fname for fname in test_dir.iterdir()]
model = models.CellposeModel(gpu=True, pretrained_model='../input/scs-cellpose-weights/SCS_CP_16_1600')

ids, masks = [],[]
for fn in test_files:
    preds, flows, _ = model.eval(io.imread(str(fn)), diameter=19, channels=[0,0], augment=True, resample=True)
    for i in range (1, preds.max() + 1):
        ids.append(fn.stem)
        masks.append(rle_encode(preds == i))
        
pd.DataFrame({'id':ids, 'predicted':masks}).to_csv('submission.csv', index=False)


In [ ]:
!python run.py

In [ ]:
import pandas as pd
pd.read_csv('submission.csv').head()